# Imports

In [1]:
import polars as pl

In [2]:
def write_txt(data: pl.DataFrame, col: str, filename: str):
    """
    Dump a Polars Series in a txt file.
    
    Used to create txt files of permnos/gvkeys/cusips to then put into WRDS queries. 
    It only exports the unique, non-null values of the column
    
    Parameters
    ----------
    data: pl.DataFrame
        The DataFrame to take data from
    col: str
        The name of the column to export.
    filename: str
        The name of the txt file (including the directory instructions).

    """
    items  = data.filter(pl.col(col).is_not_null()).select(col).unique()
    with open(f'../../data/wrds_queries_files/{filename}', 'w') as file:
        for item in items.iter_rows(): 
            file.write(str(item[0]) + '\n')

# Load Bond Returns Data

For the definitions of the features, refer to the pdf in the `docs` folder

Load the data from *openbondassetpricing.com*

In [9]:
def load_bond_data(fetch_online: bool = False) -> pl.DataFrame: 
    """
    Returns WRDS bond data as pandas dataframe.
    
    Parameters
    ----------
    fetch_online: bool, optional. Whether to fetch online bond data or to take it from the data folder.

    Returns
    -------
    pd.DataFrame: bond data
    """

    if fetch_online: 
        _url = "https://openbondassetpricing.com/wp-content/uploads/2024/04/WRDS_MMN_Corrected_Data_2024.csv"
    else: 
        _url = "../../data/unprocessed_data/openbondassetpricing_bond_data_mmn_adj.csv"

    # url to fetch
    bond_data = pl.read_csv(_url, try_parse_dates=True)

    return bond_data

In [10]:
data_obap = load_bond_data(fetch_online=False)

data_obap.head(1)

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,mom6_1,ltrev48_12,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,DURATION,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,bond_type,principal_amt,bondpar_mil
i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64
0,2002-08-31,"""000336AE7""",-0.056089,-0.087082,-0.081158,-0.054689,-0.008212,-0.009612,-0.023343,8.0,0.043019,null,0.073689,100000.0,100000.0,97.693,0.97693,70.033333,4.585289,null,6211,0.0,0.009058,0.023939,3.597388,null,2.0,0.0,1.0,0.064316,0.032526,102.092,103.75346,4.63593,26.445978,null,9.7693e6,1.02092e7,6.875,"""CDEB""",1000.0,100.0


Loading the data from the **WRDS Bond Returns** Query

In [12]:
dtypes = {
    'AMOUNT_OUTSTANDING': pl.Float64,
    'OFFERING_AMT': pl.Float64
}

data_wrds = pl.read_csv("../../data/unprocessed_data/wrds_bond_data.csv.gz", try_parse_dates=True, dtypes=dtypes)

data_wrds.head(2)

DATE,ISSUE_ID,CUSIP,bond_sym_id,bsym,ISIN,company_symbol,BOND_TYPE,SECURITY_LEVEL,CONV,OFFERING_DATE,OFFERING_AMT,OFFERING_PRICE,PRINCIPAL_AMT,MATURITY,TREASURY_MATURITY,COUPON,DAY_COUNT_BASIS,DATED_DATE,FIRST_INTEREST_DATE,LAST_INTEREST_DATE,NCOUPS,AMOUNT_OUTSTANDING,R_SP,R_MR,R_FR,N_SP,N_MR,N_FR,RATING_NUM,RATING_CAT,RATING_CLASS,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,TMT,REMCOUPS,DURATION,DEFAULTED,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE
date,i64,str,str,str,str,str,str,str,i64,date,f64,f64,i64,date,str,f64,str,date,date,date,i64,f64,str,str,str,i64,i64,i64,i64,str,str,date,str,str,str,f64,str,f64,f64,f64,i64,i64,date,f64,f64,i64,str,str,str,f64,i64,f64,str,str,str,str,str
2002-07-31,2,"""000361AB1""","""AIR.GA""",null,"""US000361AB18""","""AIR""","""CDEB""","""SEN""",0,1993-10-12,50000.0,100.0,1000,2003-10-15,"""10 YEAR""",7.25,"""30/360""",1993-10-15,1994-04-15,2003-04-15,2,50000.0,"""BBB""","""BAA3""","""BB+""",9,10,11,9,"""BBB""","""0.IG""",2002-07-05,"""$2,000""","""$2,056""",null,4.936,"""4.827%""",102.8,null,null,null,99,2002-10-15,0.0,2.13,1,null,null,null,1.23,3,1.13,"""N""",null,null,null,null
2002-08-31,2,"""000361AB1""","""AIR.GA""",null,"""US000361AB18""","""AIR""","""CDEB""","""SEN""",0,1993-10-12,50000.0,100.0,1000,2003-10-15,"""10 YEAR""",7.25,"""30/360""",1993-10-15,1994-04-15,2003-04-15,2,50000.0,"""BBB""","""BAA3""","""BB+""",9,10,11,9,"""BBB""","""0.IG""",2002-08-30,"""$35,000""","""$35,309""",null,6.3985,"""4.386%""",103.1,103.1,103.1,1,100,2002-10-15,0.0,2.74,1,""".8709%""",null,null,1.14,3,1.05,"""N""",null,null,null,null


Merge the OBAP and WRDS data together, keeping OBAP as the main DataFrame

In [13]:
merged = data_obap.join(data_wrds, how='left', left_on=['cusip', 'date'], right_on=['CUSIP', 'DATE'])

merged.head()

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,mom6_1,ltrev48_12,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,DURATION,CONVEXITY,CS_6M_DELTA,…,NCOUPS,AMOUNT_OUTSTANDING,R_SP,R_MR,R_FR,N_SP,N_MR,N_FR,RATING_NUM,RATING_CAT,RATING_CLASS,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,TMT,REMCOUPS,DURATION_right,DEFAULTED,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE
i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,f64,str,str,str,i64,i64,i64,i64,str,str,date,str,str,str,f64,str,f64,f64,f64,i64,i64,date,f64,f64,i64,str,str,str,f64,i64,f64,str,str,str,str,str
0,2002-08-31,"""000336AE7""",-0.056089,-0.087082,-0.081158,-0.054689,-0.008212,-0.009612,-0.023343,8.0,0.043019,null,0.073689,100000.0,100000.0,97.693,0.97693,70.033333,4.585289,null,6211,0.0,0.009058,0.023939,3.597388,null,2.0,0.0,1.0,0.064316,0.032526,102.092,103.75346,4.63593,26.445978,null,…,2,100000.0,"""BBB+""","""BAA2""","""BBB+""",8,9,8,8,"""BBB""","""0.IG""",2002-08-30,"""$119,000""","""$118,742""",null,6.918,"""7.369%""",97.7,97.7,97.7,1,44,2002-12-01,0.0,1.71,1,"""-.8212%""",null,"""-.8212%""",5.84,12,4.59,"""N""",null,null,null,null
5794,2002-09-30,"""000336AE7""",0.050663,0.057193,0.054383,0.052063,-0.054689,-0.056089,-0.081158,8.0,0.063356,null,0.087596,100000.0,100000.0,91.691494,0.916915,69.033333,4.437498,null,6211,-0.008212,-0.037939,-0.009819,34.628775,null,2.0,0.0,1.0,0.075301,0.049044,97.0,99.138889,4.514543,25.309944,null,…,2,100000.0,"""BBB+""","""BAA2""","""BBB+""",8,9,8,8,"""BBB""","""0.IG""",2002-09-30,"""$618,000""","""$568,962""","""0.77%""",8.6558,"""8.760%""",91.7,91.7,91.7,1,45,2002-12-01,0.0,2.28,1,"""-5.469%""","""-5.469%""","""-5.469%""",5.75,12,4.44,"""N""",null,null,null,null
11572,2002-10-31,"""000336AE7""",0.07968,0.098591,0.095571,0.08088,0.052063,0.050663,0.054383,10.0,0.052394,null,0.077672,100000.0,100000.0,96.0,0.96,68.0,4.404651,null,6211,-0.062452,-0.027926,0.080031,null,null,1.0,0.0,1.0,0.074274,0.045408,97.5,100.288197,4.428541,24.496262,null,…,2,100000.0,"""BBB-""","""BAA2""","""BBB+""",10,9,8,10,"""BBB""","""0.IG""",2002-10-31,"""$11,510,000""","""$11,001,450""",null,7.855,"""7.767%""",96.0,96.0,96.0,1,46,2002-12-01,0.0,2.86,1,"""5.206%""","""5.206%""","""5.206%""",5.67,12,4.4,"""N""",null,null,null,null
17371,2002-11-30,"""000336AE7""",null,null,null,null,0.08088,0.07968,0.095571,10.0,0.030568,null,0.061289,100000.0,100000.0,103.431,1.03431,67.0,4.40485,null,6211,-0.01364,-0.020827,0.029465,-33.329326,null,3.0,0.0,1.0,0.068748,0.039786,100.0,103.361111,4.374959,23.981731,null,…,2,100000.0,"""BBB-""","""BAA2""","""BBB+""",10,9,8,10,"""BBB""","""0.IG""",2002-11-27,"""$38,526,000""","""$33,498,709""","""0.73%""",9.9963,"""6.129%""",103.4,null,103.4,1,47,2002-12-01,0.0,3.43,1,"""8.088%""",null,"""8.088%""",5.58,12,4.4,"""N""",null,null,null,null
23064,2002-12-31,"""000336AE7""",null,null,null,null,null,null,null,10.0,0.051327,null,0.07661,100000.0,100000.0,null,null,65.966667,4.401693,null,6211,0.066137,-0.032151,0.023574,null,null,1.0,0.0,1.0,0.107563,0.079508,84.25,84.440971,4.321904,22.989123,null,…,2,100000.0,"""BBB-""","""BAA2""","""BBB+""",10,9,8,10,"""BBB""","""0.IG""",2002-12-13,"""$4,090,000""","""$3,435,014""","""1.66%""",10.8358,"""7.661%""",96.6,null,null,1,48,2003-06-01,3.44,0.57,1,"""-5.884%""",null,null,5.5,11,4.4,"""N""",null,null,null,null


In [14]:
bond_data = merged

# Data Cleaning on Bond Data

For all the characteristics except for maturity, I get the CUSIPs of the bonds and remove all the bond observations, not only the observations that have that characteristics. For maturity, I remove all observations where the Time To Maturity is over 30 years. 

In [8]:
bond_data = merged

In [15]:
# examin NaNs
bond_data.null_count()

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,mom6_1,ltrev48_12,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,DURATION,CONVEXITY,CS_6M_DELTA,…,NCOUPS,AMOUNT_OUTSTANDING,R_SP,R_MR,R_FR,N_SP,N_MR,N_FR,RATING_NUM,RATING_CAT,RATING_CLASS,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,TMT,REMCOUPS,DURATION_right,DEFAULTED,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,354192,357661,357661,354192,349654,349654,353167,0,8971,190379,8514,0,0,209439,209439,0,8971,355758,0,93805,727484,189174,257059,721563,182998,0,0,101778,101778,101778,101778,101778,101778,289731,…,81,706,667,14232,88098,12649,54901,436623,3599,81,6642,81,81,81,164095,35999,8594,81,600517,209553,19012,89255,1331,81,81,81,76364,812687,349769,81,81,9051,81,1350454,1350454,1355682,1355886


## Merge duration data

Merge duration data, since it is the same measure, and by doing so we can reduce the amount of NaNs

In [16]:
bond_data.select(['DURATION', 'duration']).null_count()

DURATION,duration
u32,u32
101778,8971


In [17]:
bond_data = bond_data.with_columns((pl.col('duration').fill_null(pl.col('DURATION'))).alias('duration'))
bond_data = bond_data.drop('DURATION')

bond_data.head(1)

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,mom6_1,ltrev48_12,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,…,NCOUPS,AMOUNT_OUTSTANDING,R_SP,R_MR,R_FR,N_SP,N_MR,N_FR,RATING_NUM,RATING_CAT,RATING_CLASS,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,TMT,REMCOUPS,DURATION_right,DEFAULTED,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE
i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,f64,str,str,str,i64,i64,i64,i64,str,str,date,str,str,str,f64,str,f64,f64,f64,i64,i64,date,f64,f64,i64,str,str,str,f64,i64,f64,str,str,str,str,str
0,2002-08-31,"""000336AE7""",-0.056089,-0.087082,-0.081158,-0.054689,-0.008212,-0.009612,-0.023343,8.0,0.043019,null,0.073689,100000.0,100000.0,97.693,0.97693,70.033333,4.585289,null,6211,0.0,0.009058,0.023939,3.597388,null,2.0,0.0,1.0,0.064316,0.032526,102.092,103.75346,26.445978,null,9.7693e6,…,2,100000.0,"""BBB+""","""BAA2""","""BBB+""",8,9,8,8,"""BBB""","""0.IG""",2002-08-30,"""$119,000""","""$118,742""",null,6.918,"""7.369%""",97.7,97.7,97.7,1,44,2002-12-01,0.0,1.71,1,"""-.8212%""",null,"""-.8212%""",5.84,12,4.59,"""N""",null,null,null,null


In [18]:
bond_data.select('duration').null_count()

duration
u32
849


## Delete the colums we don't need

Note for the rating columns: the `rating` variable has actually the least amount of NaNs, but I did not check if the NaNs align between different providers (or, e.g., if there are some bonds which are not rated by S&P but are rated by Moodie's). In that case, we can merge them and reduce the NaNs, although we need to convert to a standard rating scale

In [20]:
bond_data.select('CONV').unique()

CONV
i64
null
0


In [21]:
useless_cols = [
    'company_symbol', # we do not need it
    'TREASURY_MATURITY', # we do not need it
    'CONV', # convertible bonds (we already removed all convertibles) 
    'mom6_1', # 6m momentum (we don't need it, we are going to calculate excess credit return ourselves) 
    'ltrev48_12', # sum of bond returns over 48months (momentum) 
    'TMT', # time to maturity in years (we have it in months in tmt) and TMT has NaNs while tmt does not
    'DATE', # has NaNs while date does not
    'CUSIP', # has Nans while cusip does not
    'COUPON', # repeated in coupon and COUPON has NaNs while coupon does not
]
useless_rating_cols = ['R_SP', 'R_MR', 'R_FR', 'RATING_NUM', 'RATING_CAT', 'RATING_CLASS']

bond_data = bond_data.drop(useless_cols + useless_rating_cols)

## Filling the data which is constant for every CUSIP

Some data is missing in some dates observations, but it is the same for all bonds. So, we fill the `null` values using the other date observations

In [23]:
cols_to_fill = [
    'DATED_DATE', 
    'OFFERING_DATE', 
    'DAY_COUNT_BASIS', 
    'MATURITY', 
    'NCOUPS', 
    'FIRST_INTEREST_DATE', 
    'DEFAULTED',
    'ind_num_17',
]

In [24]:
bond_data.select(cols_to_fill).null_count()

DATED_DATE,OFFERING_DATE,DAY_COUNT_BASIS,MATURITY,NCOUPS,FIRST_INTEREST_DATE,DEFAULTED,ind_num_17
u32,u32,u32,u32,u32,u32,u32,u32
81,176,178,81,81,81,81,355758


In [25]:
# fill all DATED_DATE, OFFERING_DATE, MATURITY, and DAY_COUNT_BASIS since they are all the same for each issue
grouped = bond_data.group_by('cusip')

def fill_dates(group): 
    return group.with_columns(group.select(cols_to_fill).fill_null(strategy='forward').fill_null(strategy='backward'))
    
filled_data = grouped.map_groups(fill_dates).drop_nulls(subset=['DATED_DATE', 'OFFERING_DATE', 'MATURITY', 'NCOUPS'])
n_dropped_cusips = bond_data.n_unique('cusip') - filled_data.n_unique('cusip')

print(f'Removed {n_dropped_cusips} CUSIPs')
bond_data = filled_data

Removed 2 CUSIPs


In [26]:
bond_data.select(cols_to_fill + ['coupon', 'date']).null_count()

DATED_DATE,OFFERING_DATE,DAY_COUNT_BASIS,MATURITY,NCOUPS,FIRST_INTEREST_DATE,DEFAULTED,ind_num_17,coupon,date
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,97,0,0,0,0,226100,0,0


## Remove defaulted bonds

Remove the bonds which defaulted (`DEFAULTED=="N"`), since we don't want to consider distressed debt. The column is the same for each cusip and indicates if the bond defaulted at any time from offering date through maturity

In [27]:
bond_data.select('DEFAULTED').unique()

DEFAULTED
str
"""N"""
"""Y"""


In [30]:
mask_defaulted = pl.col('DEFAULTED') == 'Y'
cusips_defaulted = bond_data.filter(mask_defaulted).select('cusip').unique()
print(f'Found {len(cusips_defaulted)} defaulted bonds')

bond_data = bond_data.filter(~pl.col('cusip').is_in(cusips_defaulted))

Found 0 defaulted bonds


In [31]:
bond_data.select('DEFAULTED').unique()

DEFAULTED
str
"""N"""


In [32]:
# now drop the defaulted column 
bond_data = bond_data.drop('DEFAULTED')

### Remove bonds which have a default date, even if they were not marked as defaulted initially

In [56]:
# remove bonds that have a default date different from null
mask_defaulted = (pl.col('DEFAULT_DATE').is_not_null()) | (pl.col('DEFAULT_TYPE').is_not_null())
cusips_defaulted = bond_data.filter(mask_defaulted).select('cusip').unique()

# which are this percentage of the total
pct_total = bond_data.filter(mask_defaulted).shape[0] / bond_data.shape[0]
print(f'{pct_total*100:.2f}% of bonds have effectively defaulted')

2.61% of bonds have effectively defaulted


In [57]:
bond_data = bond_data.filter(~pl.col('cusip').is_in(cusips_defaulted))

### Remove bonds with prices below 10, which have effectively defaulted

In [59]:
# remove bonds whose price is < 10
mask_price_below_10 = pl.col('bondprc') < 10
cusips_price_below_10 = bond_data.filter(mask_price_below_10).select('cusip').unique()

# this is gonna remove this percentage of observations
print(f'we are going to remove {bond_data.filter(mask_price_below_10).shape[0] / bond_data.shape[0] * 100:.2f}% of the observations')

we are going to remove 0.00% of the observations


In [60]:
bond_data = bond_data.filter(~pl.col('cusip').is_in(cusips_price_below_10))

In [63]:
bond_data.select(['bondprc', 'bond_ret']).max()

bondprc,bond_ret
f64,f64
188.919,4.109004


In [64]:
bond_data.filter(pl.col('bond_ret') > 2) 

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,…,OFFERING_AMT,OFFERING_PRICE,PRINCIPAL_AMT,MATURITY,DAY_COUNT_BASIS,DATED_DATE,FIRST_INTEREST_DATE,LAST_INTEREST_DATE,NCOUPS,AMOUNT_OUTSTANDING,N_SP,N_MR,N_FR,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DURATION_right,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE
i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,i64,date,str,date,date,date,i64,f64,i64,i64,i64,date,str,str,str,f64,str,f64,f64,f64,i64,i64,date,f64,f64,i64,str,str,str,i64,f64,str,str,str,str
41066,2003-03-31,"""124845AF5""",0.073916,0.065186,0.0741,0.074916,4.109004,4.108004,1.006014,7.0,0.028725,-0.197507,0.066983,52000.0,100000.0,104.75,1.0475,250.666667,10.645989,null,4833,-0.084855,1075.668501,0.053745,11.0,0.0,1.0,0.069402,0.029285,102.0,104.909383,166.371029,-0.230125,5.447e6,5.304e6,7.125,…,100000.0,99.94,1000,2023-11-01,"""30/360""",1993-11-01,1994-05-01,2023-05-01,2,52000.0,7,7,null,2003-03-31,"""$1,380,000""","""$1,516,530""","""2.92%""",6.2612,"""6.698%""",104.8,104.8,104.8,1,106,2003-05-01,0.0,2.93,1,"""100.0%""","""100.0%""","""100.0%""",42,10.65,null,null,null,null
401003,2009-04-30,"""767754AD6""",0.399284,0.409103,0.403951,0.399284,2.11437,2.11427,1.006113,18.0,0.285359,-0.260136,0.297752,200000.0,200000.0,46.419635,0.464196,52.266667,2.958653,15.0,5912,1.494702,148.198838,0.172771,20.0,1.0,1.0,0.287631,0.274718,47.8813,49.294494,11.682567,-0.272488,9.283927e6,9.57626e6,6.875,…,200000.0,99.567,1000,2013-08-15,"""30/360""",1993-08-15,1994-02-15,2013-02-15,2,200000.0,18,20,20,2009-04-30,"""$24,704,000""","""$8,099,475""","""11.15%""",40.7088,"""29.78%""",46.4,46.4,46.4,1,182,2009-08-15,0.0,1.39,1,"""100.0%""","""100.0%""","""100.0%""",9,2.96,null,null,null,null
400039,2009-04-30,"""552953AE1""",0.856278,0.85559,0.855746,0.856278,2.62259,2.62249,0.998527,14.0,0.705536,0.670527,0.710783,400000.0,400000.0,42.001685,0.420017,21.4,1.143808,17.0,7993,0.515159,-13.468802,0.473948,19.0,1.0,1.0,0.995276,0.99027,30.4842,32.554686,1.450528,1.029699,1.6800674e7,1.219368e7,8.375,…,400000.0,100.0,1000,2011-02-01,"""30/360""",2001-01-23,2001-08-01,2010-08-01,2,400000.0,14,20,21,2009-04-30,"""$152,760,000""","""$45,941,559""","""6.62%""",98.3948,"""71.08%""",42.0,42.0,42.0,1,92,2009-08-01,0.0,2.02,1,"""100.0%""","""100.0%""","""100.0%""",4,1.14,null,null,null,null


## Remove extra long maturities (>30 years) and bonds maturing in <1y

In [33]:
mask_long_maturities = pl.col('tmt') / 12 < 30
bond_data.filter(mask_long_maturities).shape[0] / bond_data.shape[0]

0.9790509657974658

In [34]:
bond_data = bond_data.filter(mask_long_maturities)

In [35]:
mask_short_maturities = pl.col('tmt') > 12
bond_data.filter(mask_short_maturities).shape[0] / bond_data.shape[0]

1.0

In [36]:
bond_data = bond_data.filter(mask_short_maturities)

## Remove zero coupon bonds

In [40]:
mask_zcb = pl.col('NCOUPS') == 0
cusips_zcb = bond_data.filter(mask_zcb).select('cusip').unique()

print(f'found {len(cusips_zcb)} ZCB bonds ({len(cusips_zcb) / bond_data.n_unique('cusip') * 100:.2f}%)')

bond_data = bond_data.filter(~pl.col('cusip').is_in(cusips_zcb))

found 0 ZCB bonds (0.00%)


## Fix the outliers where the DATED_DATE is after the first coupon

In [41]:
bond_data.select(pl.col('FIRST_INTEREST_DATE')).null_count()

FIRST_INTEREST_DATE
u32
0


In [42]:
bond_data_fixed = bond_data.with_columns(
    DATED_DATE = pl.when((pl.col('FIRST_INTEREST_DATE') < pl.col('DATED_DATE'))).then(pl.col('OFFERING_DATE')).otherwise(pl.col('DATED_DATE'))
)

bond_data = bond_data_fixed
to_fix_mask = pl.col('FIRST_INTEREST_DATE') < pl.col('OFFERING_DATE')

while bond_data.filter(to_fix_mask).n_unique('cusip') > 0:      
    bond_data = bond_data.with_columns(
        FIRST_INTEREST_DATE=pl
            .when(to_fix_mask & (pl.col('NCOUPS') == 2)).then(pl.col('FIRST_INTEREST_DATE').dt.offset_by('6mo'))
            .when(to_fix_mask & (pl.col('NCOUPS') == 4)).then(pl.col('FIRST_INTEREST_DATE').dt.offset_by('3mo'))
            .when(to_fix_mask & (pl.col('NCOUPS') == 12)).then(pl.col('FIRST_INTEREST_DATE').dt.offset_by('1mo'))
            .otherwise(pl.col('FIRST_INTEREST_DATE'))
    )

In [43]:
bond_data.select(pl.col('FIRST_INTEREST_DATE')).null_count()

FIRST_INTEREST_DATE
u32
0


In [44]:
bond_data.filter(pl.col('FIRST_INTEREST_DATE') < pl.col('OFFERING_DATE')).n_unique('cusip')

0

## Filter to bonds where we have a minimum number of observations

In [45]:
bond_data.shape

(1357235, 84)

In [46]:
# at least 36 months of observations (bond returns) to work on
cusips_min_obs_thres = bond_data.group_by('cusip').n_unique().filter(pl.col('bond_ret') >= 36).select('cusip')
total_cusips = bond_data.n_unique('cusip')
print(f'we have {len(cusips_min_obs_thres)} CUSIPs we can work on out of {total_cusips} ({len(cusips_min_obs_thres) / total_cusips * 100:.2f}%)')


we have 11716 CUSIPs we can work on out of 25660 (45.66%)


In [47]:
bond_data = bond_data.filter(pl.col('cusip').is_in(cusips_min_obs_thres))

In [48]:
bond_data.shape

(1008198, 84)

## Abnormal Prices

Some bonds have abnormal prices, way above par. We remove them. 

In [51]:
mask_price_above_200 = pl.col('bondprc') > 200

bond_data.filter(mask_price_above_200).select('cusip').unique()

cusip
str
"""767754BD5"""
"""277461BD0"""
"""27746QAE4"""
"""85590AAD6"""
"""364760AG3"""
"""63858REN7"""
"""826428AH7"""
"""718592AB4"""


In [52]:
cusips_abnormal_prices = bond_data.filter(mask_price_above_200).select('cusip').unique()
print(f'found {len(cusips_abnormal_prices)} bonds with price above 200')
bond_data = bond_data.filter(~pl.col('cusip').is_in(cusips_abnormal_prices))

found 8 bonds with price above 200


In [53]:
bond_data.filter(mask_price_above_200).select('cusip').unique()

cusip
str


## Looking at the data more closely

In [65]:
bond_data.null_count()

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,…,OFFERING_AMT,OFFERING_PRICE,PRINCIPAL_AMT,MATURITY,DAY_COUNT_BASIS,DATED_DATE,FIRST_INTEREST_DATE,LAST_INTEREST_DATE,NCOUPS,AMOUNT_OUTSTANDING,N_SP,N_MR,N_FR,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DURATION_right,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,161319,163556,163556,161319,158090,158090,160370,0,4743,71495,4657,0,0,89083,89083,0,210,90324,0,64917,99157,382015,61981,0,0,29515,29515,29515,29515,29515,110641,89083,29515,0,…,6,164457,6,0,94,0,0,6,0,238,5699,33185,264310,6,6,6,74022,25945,4662,6,356602,89111,7573,45266,6,6,6,6,18967,510409,158120,6,4748,978379,978379,978379,978379


In [66]:
bond_data.group_by('cusip').agg(pl.all().null_count()).sort('bond_ret', descending=True)

cusip,,date,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,…,OFFERING_AMT,OFFERING_PRICE,PRINCIPAL_AMT,MATURITY,DAY_COUNT_BASIS,DATED_DATE,FIRST_INTEREST_DATE,LAST_INTEREST_DATE,NCOUPS,AMOUNT_OUTSTANDING,N_SP,N_MR,N_FR,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DURATION_right,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""278058AW2""",0,0,166,166,166,166,165,165,165,0,0,20,0,0,0,104,104,0,0,0,0,109,151,165,105,0,0,51,51,51,51,51,94,104,51,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,88,0,0,0,186,104,0,0,0,0,0,0,19,217,165,0,0,220,220,220,220
"""505588AY9""",0,0,162,162,162,162,162,162,162,0,0,28,0,0,0,104,104,0,0,202,0,160,187,185,165,0,0,118,118,118,118,118,168,104,118,0,…,0,0,0,0,0,0,0,0,0,0,0,0,51,0,0,0,41,0,0,0,166,104,1,0,0,0,0,0,23,197,162,0,0,202,202,202,202
"""029163AD4""",0,0,160,160,160,160,160,160,160,0,0,27,0,0,0,99,99,0,0,0,0,87,126,160,82,0,0,44,44,44,44,44,80,99,44,0,…,0,210,0,0,0,0,0,0,0,0,0,0,58,0,0,0,64,82,0,0,172,99,0,0,0,0,0,0,23,204,160,0,0,210,210,210,210
"""49337WAC4""",0,0,161,161,161,161,160,160,160,0,0,27,0,0,0,104,104,0,0,0,0,111,147,163,107,0,0,59,59,59,59,59,101,104,59,0,…,0,0,0,0,0,0,0,0,0,0,0,0,42,0,0,0,90,0,0,0,171,104,0,0,0,0,0,0,28,199,160,0,0,205,205,205,205
"""078167BA0""",0,0,159,159,159,159,159,159,159,0,0,31,0,0,0,101,101,0,0,207,0,108,143,159,103,0,0,57,57,57,57,57,102,101,57,0,…,0,0,0,0,0,0,0,0,0,0,0,115,0,0,0,0,73,10,0,0,173,101,0,0,0,0,0,0,20,202,159,0,0,207,207,207,207
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""852060AC6""",0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,4,0,0,0,63,63,63,63
"""700690AH3""",0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,…,0,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,26,0,0,0,42,42,42,42
"""68389XAW5""",0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,18,0,0,0,0,0,0,0,0,1,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,20,0,0,1,0,0,0,0,0,36,0,0,0,94,94,94,94


In [67]:
# percentage of nan values for the bond returns
(bond_data.group_by('cusip').agg([pl.all().null_count(), pl.col('bond_ret').len().alias('count')])
    .select('cusip', 'date', 'bond_ret', 'bond_ret_t+1', 'count')
    .with_columns(pl.all().exclude('cusip', 'count') / pl.col('count'))
    .sort('bond_ret', descending=True)
    .filter(pl.col('bond_ret') < 0.5)
 )

cusip,date,bond_ret,bond_ret_t+1,count
str,f64,f64,f64,u32
"""976826BE6""",0.0,0.49711,0.49711,173
"""039483AH5""",0.0,0.496933,0.496933,163
"""740816AD5""",0.0,0.496503,0.496503,143
"""706451BD2""",0.0,0.496403,0.496403,139
"""755111AF8""",0.0,0.495726,0.5,234
…,…,…,…,…
"""655664AM2""",0.0,0.0,0.020833,48
"""46625HRS1""",0.0,0.0,0.013158,76
"""02003MBQ6""",0.0,0.0,0.020833,48


This may still a bit high, but it could further reduce our investable universe (which is already at 45% something of the initial total)

# Merging with Equity Data

We will be performing several `join_asof`, where data needs to be first sorted by date, to merge the equity data to the bond returns data we already have, and to precisely link each bond with its PERMNO

## Linking the bond CUSIP to PERMNOs using WRDS

Export the list of cusips

In [72]:
write_txt(bond_data, 'cusip', '../../data/wrds_queries_files/cusips_list.txt')

Download the csv linking cusip to permno from WRDS, and import it

In [73]:
cusip_permno_mapping = pl.read_csv('../../data/wrds_queries_files/cusips_to_permno_dataset.csv', try_parse_dates=True)

cusip_permno_mapping.head()

cusip,PERMNO,PERMCO,link_startdt,link_enddt
str,i64,i64,date,date
"""00037BAA0""",88953,41444,2012-05-03,2012-05-04
"""00037BAA0""",88953,41444,2012-05-03,2017-05-02
"""00037BAB8""",88953,41444,2012-05-03,2012-05-04
"""00037BAB8""",88953,41444,2012-05-03,2022-05-05
"""00037BAC6""",88953,41444,2012-05-03,2023-05-22


An example of bond which was associated with two PERMNOs across its existence

In [76]:
cusip_permno_mapping.filter(pl.col('cusip') == "87612BAP7")

cusip,PERMNO,PERMCO,link_startdt,link_enddt
str,i64,i64,date,date
"""87612BAP7""",12476,53621,2018-03-19,2021-05-17
"""87612BAP7""",91823,51812,2014-05-21,2016-02-16


Perform an asof join. The mode is backward, so that each cusip is associated to the permno only after the link_startd date

In [77]:
cusip_permno_mapping = cusip_permno_mapping.sort('link_startdt')
bond_data = bond_data.sort('date')

bond_data = bond_data.join_asof(
    cusip_permno_mapping,
    left_on='date',
    right_on='link_startdt',
    by_left=['cusip'],
    by_right=['cusip'],
    strategy='backward'
)

In [79]:
bond_data.group_by('cusip').n_unique().select('PERMNO').to_series().value_counts()

PERMNO,count
u32,u32
1,9853
4,1
2,1321
3,149


Let's see how many bonds were not matched

In [80]:
bond_data.filter(pl.col('PERMNO').is_null()).n_unique('cusip') / bond_data.n_unique('cusip')

0.09510773578240904

In [81]:
bond_data.group_by('cusip').agg(pl.col('PERMNO').null_count()).sort('PERMNO', descending=True)

cusip,PERMNO
str,u32
"""595620AB1""",237
"""751277AQ7""",205
"""129466AM0""",195
"""59562VAM9""",193
"""59562EAH8""",187
…,…
"""743315AN3""",0
"""58013MEL4""",0
"""404119BQ1""",0


Try to forward fill the PERMNO column

In [82]:
# fill all DATED_DATE, OFFERING_DATE, MATURITY, and DAY_COUNT_BASIS since they are all the same for each issue
grouped = bond_data.group_by('cusip')

def fill_dates(group): 
    return group.with_columns(group.select('PERMNO').fill_null(strategy='forward'))

filled_data = grouped.map_groups(fill_dates)
bond_data = filled_data

In [83]:
bond_data.group_by('cusip').agg(pl.col('PERMNO').null_count()).sort('PERMNO', descending=True)

cusip,PERMNO
str,u32
"""595620AB1""",237
"""751277AQ7""",205
"""129466AM0""",195
"""59562VAM9""",193
"""59562EAH8""",187
…,…
"""247361ZU5""",0
"""264399EH5""",0
"""69073TAJ2""",0


It didn't do much, but most likely these are bonds for which the PERMNO wasn't found at all

Export the list of unique permnos, and switch over to WRDS to get the fundamental data we need

In [85]:
# export the list of permnos to make the query to WRDS 
write_txt(bond_data, 'PERMNO', '../../data/wrds_queries_files/companies_permnos.txt')

## Load the financial ratios from WRDS

In [93]:
finratios = pl.read_csv('../../data/unprocessed_data/wrds_finratios.csv', try_parse_dates=True)

finratios.head()

gvkey,permno,adate,qdate,public_date,capital_ratio,debt_ebitda,de_ratio,intcov_ratio,cash_ratio
i64,i64,date,date,date,f64,f64,f64,f64,f64
11903,10025,1999-10-31,1999-10-31,2000-01-31,0.808,4.371,6.705,1.24,0.019
11903,10025,1999-10-31,1999-10-31,2000-02-29,0.808,4.371,6.705,1.24,0.019
11903,10025,1999-10-31,2000-01-31,2000-03-31,0.807,4.912,6.78,1.053,0.022
11903,10025,1999-10-31,2000-01-31,2000-04-30,0.807,4.912,6.78,1.053,0.022
11903,10025,1999-10-31,2000-01-31,2000-05-31,0.807,4.912,6.78,1.053,0.022


In [88]:
finratios.n_unique('gvkey')

1145

In [89]:
finratios.null_count()

gvkey,permno,adate,qdate,public_date,capital_ratio,debt_ebitda,de_ratio,intcov_ratio,cash_ratio
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,217,0,0,325,1320,249,16969,40118


In [90]:
finratios = finratios.sort('public_date')
bond_data = bond_data.sort('date')

bond_data = bond_data.join_asof(
    finratios.select(pl.all().exclude(['adate', 'qdate'])),
    strategy='backward',
    left_on='date',
    right_on='public_date',
    by_left='PERMNO',
    by_right='permno'
)

## Short Interest Data

Get the gvkeys of the bond issuers, and head over to WRDS to get the Short Interest Data

In [91]:
gvkeys = finratios.select('gvkey').unique()

write_txt(finratios, 'gvkey', '../../data/wrds_queries_files/companies_gvkey.txt')

Load Short Interest Data

In [94]:
short_interest = pl.read_csv('../../data/unprocessed_data/wrds_short_interest_data.csv', try_parse_dates=True)

short_interest.head()

gvkey,iid,shortint,shortintadj,datadate,splitadjdate,tic,cusip
i64,i64,f64,f64,date,date,str,str
105365,1,3.687424e6,3.687424e6,2020-06-15,2020-06-30,"""AMT""","""03027X100"""
105365,1,2.3236144e7,2.3236144e7,2003-04-15,2003-04-30,"""AMT""","""03027X100"""
105365,1,4.347898e6,4.347898e6,2010-11-15,2010-11-30,"""AMT""","""03027X100"""
105365,1,3.635196e6,3.635196e6,2012-10-15,2012-10-31,"""AMT""","""03027X100"""
105365,1,8.965266e6,8.965266e6,2009-09-30,2009-09-30,"""AMT""","""03027X100"""


In [95]:
short_interest.null_count()

gvkey,iid,shortint,shortintadj,datadate,splitadjdate,tic,cusip
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


In [96]:
short_interest = short_interest.sort('datadate')
bond_data = bond_data.sort('date')

bond_data.join_asof(
    short_interest,
    left_on='date',
    right_on='datadate',
    by='gvkey',
    strategy='backward'
).null_count()

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,…,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DURATION_right,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE,PERMNO,PERMCO,link_startdt,link_enddt,gvkey,public_date,capital_ratio,debt_ebitda,de_ratio,intcov_ratio,cash_ratio,iid,shortint,shortintadj,datadate,splitadjdate,tic,cusip_right
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,161319,163556,163556,161319,158090,158090,160370,0,4743,71495,4657,0,0,89083,89083,0,210,90324,0,64917,99157,382015,61981,0,0,29515,29515,29515,29515,29515,110641,89083,29515,0,…,4662,6,356602,89111,7573,45266,6,6,6,6,18967,510409,158120,6,4748,978379,978379,978379,978379,55365,55365,55365,55365,160899,160899,161690,166029,161693,214566,344468,894288,894288,894288,894288,894288,894288,894288


In [97]:
bond_data.n_unique('gvkey')

1128

# Inflation Expectations

In [98]:
cpi = pl.read_csv('../../data/inflation/cpi_monthly.csv', try_parse_dates=True)
expected_cpi = pl.read_csv('../../data/inflation/expected_1y_infl.csv', try_parse_dates=True)

In [99]:
cpi.head()

DATE,CPIAUCSL
date,f64
1947-01-01,21.48
1947-02-01,21.62
1947-03-01,22.0
1947-04-01,22.0
1947-05-01,21.95


In [100]:
expected_cpi.head()

DATE,EXPINF1YR
date,f64
1982-01-01,6.3945071
1982-02-01,6.4321077
1982-03-01,6.3877317
1982-04-01,6.1406276
1982-05-01,5.4881669


In [101]:
cpi = cpi.with_columns(
    cpi_growth_yoy=pl.col('CPIAUCSL').pct_change(12)
)

cpi.tail()

DATE,CPIAUCSL,cpi_growth_yoy
date,f64,f64
2023-11-01,308.024,0.031395
2023-12-01,308.742,0.033232
2024-01-01,309.685,0.03106
2024-02-01,311.054,0.031657
2024-03-01,312.23,0.034751


In [102]:
expected_cpi = expected_cpi.with_columns(
    expected_1y_infl=pl.col('EXPINF1YR') / 100
)

In [103]:
inflation_data = (expected_cpi
                  .join(cpi, how='left', on='DATE')
                  .rename({'DATE': 'date'})
                  .select(['date', 'expected_1y_infl', 'cpi_growth_yoy'])
                  .filter(pl.col('date').dt.year() >= 2000)
                  )

inflation_data.tail()

date,expected_1y_infl,cpi_growth_yoy
date,f64,f64
2023-12-01,0.030917,0.033232
2024-01-01,0.024186,0.03106
2024-02-01,0.024283,0.031657
2024-03-01,0.020822,0.034751
2024-04-01,0.026959,null


Computing the unexpected inflation

In [104]:
inflation_data = inflation_data.with_columns(
    unexpected_inflation=(pl.col('cpi_growth_yoy').diff() - pl.col('expected_1y_infl').diff())
)

inflation_data.tail()

date,expected_1y_infl,cpi_growth_yoy,unexpected_inflation
date,f64,f64,f64
2023-12-01,0.030917,0.033232,-0.001168
2024-01-01,0.024186,0.03106,0.004559
2024-02-01,0.024283,0.031657,0.000501
2024-03-01,0.020822,0.034751,0.006555
2024-04-01,0.026959,null,null


In [105]:
inflation_data.plot(x='date', y='unexpected_inflation')

:Curve   [date]   (unexpected_inflation)

In [106]:
bond_data = bond_data.sort('date')
inflation_data = inflation_data.sort('date')

bond_data = bond_data.join_asof(
    inflation_data.select(['date', 'unexpected_inflation']),
    on='date',
    strategy='backward'
)

# Export the data

In [108]:
bond_data.head(1)

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,sic_code,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,…,N_FR,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DURATION_right,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE,PERMNO,PERMCO,link_startdt,link_enddt,gvkey,public_date,capital_ratio,debt_ebitda,de_ratio,intcov_ratio,cash_ratio,unexpected_inflation
i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,date,str,str,str,f64,str,f64,f64,f64,i64,i64,date,f64,f64,i64,str,str,str,i64,f64,str,str,str,str,i64,i64,date,date,i64,date,f64,f64,f64,f64,f64,f64
2545,2002-08-31,"""370425RG7""",-0.013904,-0.039905,-0.034585,-0.012504,0.00853,0.00713,-0.003688,8.0,0.029057,0.169377,0.056987,1.1e6,1.1e6,101.787405,1.017874,55.933333,3.87212,16.0,6141,0.000912,0.046469,0.01229,21.0,1.0,1.0,0.057698,0.029467,101.5075,104.035834,18.3204,null,1.11966146e8,1.1165825e8,6.15,…,7,2002-08-30,"""$71,946,000""","""$72,123,268""","""0.95%""",6.0859,"""5.699%""",101.8,101.8,101.8,1,34,2002-10-05,0.0,2.49,1,""".8530%""",""".8530%""",""".8530%""",10,3.87,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.004985


In [107]:
bond_data.write_parquet('../../data/final_data/bond_data_final.pq', compression='zstd', compression_level=10)